In [102]:
import sqlite3
import pandas as pd
import geopandas as gpd
import folium

### How does the availability of vegetarian and vegan dishes vary by area?

In [103]:
where_clause  = """
WHERE
(m.name LIKE '%Vegi%' OR m.name LIKE '%Veg%' 
    OR m.name LIKE '%Vleesvrij%' 
    OR m.name LIKE '%Plantaardig%' 
    OR m.name LIKE '%Groen%' 
    OR m.name LIKE '%Duurzaam%' 
    OR m.description LIKE '%veg%'  
    OR m.description LIKE '%vleesvrij%' 
    OR m.description LIKE '%plantaardig%' 
    OR m.description LIKE '%groen%' 
    OR m.description LIKE '%duurzaam%'
    OR LOWER(m.name) LIKE '%veg%' 
    OR LOWER(m.name) LIKE '%meat-free%' 
    OR LOWER(m.name) LIKE '%plant-based%' 
    OR LOWER(m.name) LIKE '%green%' 
    OR LOWER(m.name) LIKE '%sustainable%' 
    OR LOWER(m.description) LIKE '%meat-free%' 
    OR LOWER(m.description) LIKE '%plant-based%' 
    OR LOWER(m.description) LIKE '%green%' 
    OR LOWER(m.description) LIKE '%sustainable%'
    OR LOWER(m.name) LIKE '%végé%' 
    OR LOWER(m.name) LIKE '%sans viande%' 
    OR LOWER(m.name) LIKE '%à base de plantes%' 
    OR LOWER(m.name) LIKE '%vert%' 
    OR LOWER(m.name) LIKE '%durable%')
"""

In [104]:
query_vegi_d = f"""   
SELECT 
    m.name AS dish_name,
    m.restaurant_id,
    r.name AS restaurant_name,
    latitude,
    longitude,
    postal_code,
    address
FROM 
    menu_items AS m
    JOIN restaurants AS r  ON m.restaurant_id = r.id
    
{where_clause} 

GROUp BY
    restaurant_id
;

"""

In [105]:
query_vegi_T = f"""   
SELECT 
    m.name AS dish_name,
    r.name AS restaurant_name,
    latitude,
    longitude,
    address,
    city
FROM 
    menuItems AS m
    JOIN restaurants AS r  ON m.primarySlug	 = r.primarySlug
{where_clause}
    AND r.latitude != 0
    AND r.longitude != 0 
GROUp BY
    restaurant_name

;

"""

In [106]:
query_vegi_U = f"""   
SELECT 
    m.name AS Item_name,
    location__address AS address,
    location__city AS city,
    location__latitude AS latitude,
    location__longitude AS longitude,
    title AS resturant_name
    
FROM 
    menu_items AS m
    JOIN restaurants AS r ON m.restaurant_id = r.id
{where_clause}
    AND r.location__latitude != 0
    AND r.location__longitude != 0 
GROUP BY
    title
;

"""

In [107]:
takeaway_db = 'Databases/takeaway.db'
deliveroo_db ='Databases/deliveroo.db'
Uber_db ='Databases/ubereats.db'
def query_db(db, sql_query):
    ''' input: db (path to SQLite database), sql_query (str)
        output: pandas DataFrame
    '''
    conn = sqlite3.connect(db)
    with conn:
        res = pd.read_sql_query(sql_query, conn)
    if conn:
        conn.close()
    return res

In [108]:
takeaway_df = query_db(takeaway_db, query_vegi_T)
deliveroo_df = query_db(deliveroo_db, query_vegi_d)
uber_df = query_db(Uber_db, query_vegi_U)

In [109]:


# Deliveroo dataset
deliveroo_df = deliveroo_df[['restaurant_name', 'address', 'latitude', 'longitude']]
deliveroo_df.rename(columns={'restaurant_name': 'Name'}, inplace=True)
deliveroo_geometry = gpd.points_from_xy(deliveroo_df.longitude, deliveroo_df.latitude)
deliveroo_gdf = gpd.GeoDataFrame(deliveroo_df, crs='EPSG:4326', geometry=deliveroo_geometry)

# Takeaway dataset
takeaway_df = takeaway_df[['restaurant_name', 'address', 'city', 'latitude', 'longitude']]
takeaway_df.rename(columns={'restaurant_name': 'Name'}, inplace=True)
takeaway_geometry = gpd.points_from_xy(takeaway_df.longitude, takeaway_df.latitude)
takeaway_gdf = gpd.GeoDataFrame(takeaway_df, crs='EPSG:4326', geometry=takeaway_geometry)

# Uber Eats dataset
ubereats_df = uber_df[['resturant_name','address','city', 'latitude', 'longitude']]
ubereats_df.rename(columns={'resturant_name': 'Name'}, inplace=True)
ubereats_geometry = gpd.points_from_xy(ubereats_df.longitude, ubereats_df.latitude)
ubereats_gdf = gpd.GeoDataFrame(ubereats_df, crs='EPSG:4326', geometry=ubereats_geometry)

# Creating a Folium map
m = folium.Map(location=[50.85, 4.35], zoom_start=10,  control_scale = True) 

# Adding Takeaway points to the map
for _, row in takeaway_gdf.iterrows():   # _ means "ignore the index, I don't need it." it is a way to save the index (first var) without using it later
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius= 10,  # Marker size
        popup=f"Takeaway<br>Name: {row['Name']}<br>Address: {row['address']}",
        color='orange',  #
        fill=True,
        fill_color='orange',  # Fill color
        fill_opacity=1
    ).add_to(m)

# Adding Uber Eats points to the map
for _, row in ubereats_gdf.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,  #
        popup=f"Uber Eats<br>Name: {row['Name']}<br>Address: {row['address']}",
        color='black',  # 
        fill=True,
        fill_color='black',  # Fill color
        fill_opacity=0.6
    ).add_to(m)
# Adding Deliveroo points to the map 
for _, row in deliveroo_gdf.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,  # Marker size
        popup=f"Deliveroo<br>Name: {row['Name']}<br>Address: {row['address']}",
        color='lightblue',  # Border color
        fill=True,
        fill_color='lightblue',  # Fill color
        fill_opacity=0.2
    ).add_to(m)
    
m.save("Veg_map.html")


